In [1]:
using Laplacians
using SuiteSparse
using LinearAlgebra
using SparseArrays

In [2]:
struct LLcol
      row::Int64
      ptr::Int64
      val::Float64
  end

struct LLord{Tind,Tval}
    row::Tind
    next::Tind
    val::Tval
end

mutable struct LLMatOrd{Tind,Tval}
    n::Int64
    cols::Array{Tind,1}
    lles::Array{LLord{Tind,Tval},1}
end

In [3]:
function LLMatOrd(a::SparseMatrixCSC{Tval,Tind}) where {Tind,Tval}
    n = size(a,1)
    m = nnz(a)

    cols = zeros(Tind, n)
    llelems = Array{LLord{Tind,Tval}}(undef, m)

    ptr = one(Tind)

    @inbounds for i in Tind(1):Tind(n-1)
        next = zero(Tind)

        for ind in (a.colptr[i]):(a.colptr[i+1]-one(Tind))
            j = a.rowval[ind]
            if (i < j)

              v = a.nzval[ind]
              llelems[ptr] = LLord{Tind,Tval}(j, next, v)
              next = ptr
              ptr += one(Tind)

            end
        end
        cols[i] = next
    end

    return LLMatOrd{Tind,Tval}(n, cols, llelems)
end

LLMatOrd

In [4]:
a = grid2(5)
la = lap(a)
llmat = LLMatOrd(la)
llmat.cols

25-element Array{Int64,1}:
  2
  4
  6
  8
  9
 11
 13
 15
 17
 18
 20
 22
 24
 26
 27
 29
 31
 33
 35
 36
 37
 38
 39
 40
  0